## 4.2 PostgreSQL - Héritage entre tables

In [1]:
import psycopg2
import psycopg2.extras

conn = psycopg2.connect(
    host="localhost",
    database="demo",
    user="demo_owner",
    password="OODBMS")

# query result as dict
c = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

<div id="create" style="font-size:120%">
Création d'une hiérarchie de tables avec héritage
</div>

In [2]:
# Création des tables
c.execute('''
DROP TABLE IF EXISTS person CASCADE;


CREATE TABLE person OF person_type;
  
CREATE UNIQUE INDEX person_name ON person (first_name,last_name);

CREATE TABLE developer (
    login VARCHAR(32),
    pwd VARCHAR(32)
) INHERITS(person);

CREATE TABLE player (
    pseudo VARCHAR(16),
    score INTEGER
) INHERITS(person);

CREATE TABLE VIP (
    email VARCHAR(128),
    preferences TEXT
) INHERITS (player)
''')
conn.commit()

<div id="insert" style="font-size:120%">
Chacune des tables dispose des colonnes héritées :
</div>

In [30]:
# Alimentation avec quelques enregistrements
c.execute('''
TRUNCATE person;
TRUNCATE developer;
TRUNCATE player;
TRUNCATE VIP;

INSERT INTO person(first_name,last_name)
       VALUES ('Raymond', 'Deubaze');
       
INSERT INTO developer (first_name,last_name,login,pwd)
       VALUES ('Elsa', 'Plique', 'epe', 'hello');

INSERT INTO player(first_name,last_name,pseudo,score)
       VALUES ('Alain','Terrieur','atr',0);

INSERT INTO VIP (first_name,last_name,pseudo, score, email,preferences)
       VALUES ('Jean','Bombeur','jbr',0,'jbr@email.com','WOW');
''')
conn.commit()

<div id="select" style="font-size:120%">
Une requête sur la table mère parcourt l'ensemble des tables filles 
</div>

In [31]:
# liste des éléments via la classe mère
c.execute('SELECT * FROM person')
for s in c.fetchall(): print(' '.join(['{:<8}'.format(v) for v in s]))

Raymond  Deubaze 
Elsa     Plique  
Alain    Terrieur
Jean     Bombeur 


In [32]:
# liste d'éléments via une classe intermédiaire
c.execute('SELECT * FROM player')
for s in c.fetchall(): print(' '.join(['{:<8}'.format(v) for v in s]))

Alain    Terrieur atr      0       
Jean     Bombeur  jbr      0       


<div id="oid" style="font-size:120%">
PostgreSQL maintient un identifiant unique par table, qui permet de remonter au nom de la table contenant un enregistrement donné&#160;:
</div>

In [6]:
# la table pg_class
c.execute('''
SELECT p.relname, person.*
FROM person, pg_class p
WHERE person.tableoid = p.oid;
''')

for s in c.fetchall(): print(' '.join(['{:<10}'.format(v) for v in s]))

person     Raymond    Deubaze   
developer  Elsa       Plique    
player     Alain      Terrieur  
vip        Jean       Bombeur   


In [7]:
c.execute('SELECT p.relname, player.* FROM player, pg_class p WHERE player.tableoid = p.oid;')
for s in c.fetchall(): print(' '.join(['{:<8}'.format(v) for v in s]))

player   Alain    Terrieur atr      0       
vip      Jean     Bombeur  jbr      0       


<div id="select_from_only" style="font-size:120%">
Il est possible de requêter uniquement sur une table mère
</div>

In [8]:
c.execute('SELECT * FROM ONLY person')
for s in c.fetchall(): print(' '.join(['{:<8}'.format(v) for v in s]))

Raymond  Deubaze 


<div id="update_superclass" style="font-size:120%">
SELECT n'est pas la seule requête qu'il est possible d'adresser à la table mère&#160;:
</div>

In [9]:
c.execute('''SELECT p.relname, person.* FROM person, pg_class p WHERE person.tableoid = p.oid;''')
for s in c.fetchall(): print(' '.join(['{:<10}'.format(v) for v in s]))

person     Raymond    Deubaze   
developer  Elsa       Plique    
player     Alain      Terrieur  
vip        Jean       Bombeur   


In [10]:
c.execute("UPDATE person SET first_name='Anna', last_name='Conda' WHERE first_name='Elsa'")
conn.commit()

In [11]:
c.execute('''SELECT p.relname, person.* FROM person, pg_class p WHERE person.tableoid = p.oid;''')
for s in c.fetchall(): print(' '.join(['{:<10}'.format(v) for v in s]))

person     Raymond    Deubaze   
developer  Anna       Conda     
player     Alain      Terrieur  
vip        Jean       Bombeur   


<div id="caveats-update-subclass" style="font-size:120%">
Par contre, on ne peut pas modifier un attribut de la table fille via la table mère&#160;:
</div>

In [29]:
try:
    c.execute("UPDATE person SET pseudo='aca' WHERE first_name='Anna'")
    conn.commit()
except psycopg2.errors.UndefinedColumn:
    print("Colonne inconnue")
    conn.rollback()

Colonne inconnue


<div id="caveats-unique" style="font-size:120%">
Si la table mère possède des contraintes d'unicité, celles-ci n'incluent pas les tables filles...
</div>

In [20]:
# impossible de dupliquer un élément dans la table mère
try:
    c.execute("INSERT INTO person VALUES ('Raymond','Deubaze')")
    conn.commit()
except psycopg2.errors.UniqueViolation:
    print("Violation d'unicité")
    conn.rollback()

Violation d'unicité


In [23]:
# Il est possible d'insérer un homonyme dans une table fille
c.execute("INSERT INTO player VALUES ('Raymond','Deubaze','rde',0)")
conn.commit()

In [25]:
# Vérification
c.execute('''SELECT p.relname, person.* FROM person, pg_class p
             WHERE person.first_name='Raymond' AND person.tableoid = p.oid;''')
for s in c.fetchall(): print(' '.join(['{:<10}'.format(v) for v in s]))

person     Raymond    Deubaze   
player     Raymond    Deubaze   


In [28]:
conn.rollback()